In [1]:
import numpy as np
import pandas as pd
import math as mt

# import path and create needed variables

In [2]:
#path for data import 
flight_data_path=r'H:\00ml\dat2\2016\on_time_2016'
weather_data_path=r'H:\00ml\dat2\airport_data'
flight_path=r'H:\00ml\dat2\cleaned data\Flight_data\n_flight'
weather_path=r'H:\00ml\dat2\cleaned data\Weather_data'
mergedata_path=r'H:\00ml\dat2\cleaned data\mergedata\n_merge'
#crating neccessary variable list data for loop
year=['2016','2017']
month=["1","2","3","4","5","6","7","8","9","10","11","12"]
airport=["ATL","CLT","DEN","DFW","EWR","IAH","JFK","LAS","LAX","MCO","MIA","ORD","PHX","SEA","SFO"]

def round_time(x):
    a=x//100
    b=x%100
    if b>30:
        x=(a+1)*100
    elif x==2400:
        x=0000
    else:
        x=a*100
    return x

print(month)
print(year)
print(airport)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
['2016', '2017']
['ATL', 'CLT', 'DEN', 'DFW', 'EWR', 'IAH', 'JFK', 'LAS', 'LAX', 'MCO', 'MIA', 'ORD', 'PHX', 'SEA', 'SFO']


# flight data cleaning

In [28]:

for r in range(0,len(year)):
    for j in range(0,len(month)):
        #print(path+"/On_Time_On_Time_Performance_"+year[r]+"_"+month[j]+".csv")
        flight_raw_data = pd.read_csv(flight_data_path+"/On_Time_On_Time_Performance_"+year[r]+"_"+month[j]+".csv",low_memory=False)
        flight_requied_data1=flight_raw_data[['FlightDate','Quarter','Year','Month','DayofMonth','DepTime','DepDel15', 'CRSDepTime','DepDelayMinutes',
         'OriginAirportID','DestAirportID','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes','Origin','Dest']]
        pd.reset_option('mode.chained_assignment')
        with pd.option_context('mode.chained_assignment',None):
            flight_requied_data1.dropna(subset=['DepTime','ArrTime'], inplace=True)
        flight_requied_data=flight_requied_data1[flight_requied_data1.Origin.isin(airport)&flight_requied_data1.Dest.isin(airport)]
        
        flight_requied_data.reset_index(drop=True)
        print("/flight_data"+year[r]+"_"+month[j])
        flight_requied_data1.to_csv(flight_path+"/flight_data"+year[r]+"_"+month[j]+".csv",index=False)

/flight_data2016_1
/flight_data2016_2
/flight_data2016_3
/flight_data2016_4
/flight_data2016_5
/flight_data2016_6
/flight_data2016_7
/flight_data2016_8
/flight_data2016_9
/flight_data2016_10
/flight_data2016_11
/flight_data2016_12
/flight_data2017_1
/flight_data2017_2
/flight_data2017_3
/flight_data2017_4
/flight_data2017_5
/flight_data2017_6
/flight_data2017_7
/flight_data2017_8
/flight_data2017_9
/flight_data2017_10
/flight_data2017_11
/flight_data2017_12


# weather data cleaning

In [4]:
for u in range(0,len(airport)):   #iterate airport name for loop
    for r in range(0,len(year)):    #iterate year for loop
        for j in range(0,len(month)):     #iterate month for loop
            
            raw_weather_data = pd.read_json(weather_data_path+"/"+airport[u]+"/"+year[r]+"-"+month[j]+".json").T
            
            # import raw weather data for cleaning and editing 
            base_weather_data=pd.DataFrame(raw_weather_data['weather'][0])
            #extact required data inside the list
            request_data=pd.DataFrame(raw_weather_data['request'][0])
            #extact required data inside the list
            airport_name=request_data['query']
            airport_name1=airport_name[0].split(',')[0]
            #extact required airport name inside the data list
            base_weather_data=base_weather_data[['date','hourly']]
            #seperate needed columns from base_weather
            base_weather_data.loc[:,'airport']= airport_name1
            #seperate airport name from base_weather
            daily_data= pd.DataFrame()
            #create empty dataframe for append process
            for i in range (len(base_weather_data)):
                #iterate base weather length for extract nested data
                hourly_data=pd.DataFrame(base_weather_data['hourly'][i])
                #isolate hourly column nested data
                hourly_data=hourly_data[['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF',
            'WindGustKmph','tempF', 'WindChillF', 'humidity','time']].astype(float)
                #select reqired column from hourly data
                date=base_weather_data['date'][i]
                #seperate date column from base weather data frame
                hourly_data.loc[:,'date']= date
                #add date column and its value in hourly data
                hourly_data.loc[:,'airport']= airport_name1
                #add airport column and its value in hourly data
                daily_data=daily_data.append(hourly_data,ignore_index=True)
            
            print("/weather_data"+year[r]+"_"+month[j]+".csv")
            #import cleaned weather data to .csv for storage with respected name
            daily_data.to_csv(weather_path+"/"+airport[u]+"/base_weather_data"+"_"+airport[u]+"_"+year[r]+"_"+month[j]+".csv",index=False)
        

/weather_data2016_1.csv


# flight and weather data merging

In [5]:
final_file=pd.DataFrame()
for r in range(0,len(year)):    #iterate year for loop
    
    for j in range(0,len(month)):   #iterate month for loop
        
        flight_data = pd.read_csv(flight_path+"/flight_data"+year[r]+"_"+month[j]+".csv") 
        # import flight data to df by reading path
        flight_data.sort_values(by=['FlightDate', 'DepTime'], inplace=True)  #order data for visulaization
        #print(flight_data['DepTime'].head(20))
        #drop null value rows which dosen't provide needed information
        flight_data['DepTimeh']=flight_data['DepTime'].apply(round_time)
        flight_data['ArrTimeh']=flight_data['ArrTime'].apply(round_time)
        #print(flight_data['DepTimeh'].head(20))
        #convert new time data in to 24 hrs format by replacing 24:00 into 00:00 and continue time in float value to reduce coplication in merging 
        origin_flight=flight_data.drop('Dest',axis=1)#creating seperate df for origin flight data
        dest_flight=flight_data.drop('Origin',axis=1)#creating seperate df for destination flight data
        origin_data=pd.DataFrame()
        dest_data=pd.DataFrame()
        flight_weather=pd.DataFrame()
        
        #create empty data frame helpfull for merge and append operation
        for u in range(0,len(airport)):   #iterate airport name for loop
            
            weather_data = pd.read_csv(weather_path+"/"+airport[u]+"/base_weather_data"+"_"+airport[u]+"_"+year[r]+"_"+month[j]+".csv")
            # import weather data to df by reading path
            origin_weather =pd.merge(origin_flight,weather_data,how='inner',left_on=['FlightDate','Origin','DepTimeh'], right_on=['date','airport','time'])
            # merging orgin flight with weather data by common given columns and produce inersection valuea as origin data
            dest_weather =pd.merge(dest_flight,weather_data,how='inner',left_on=['FlightDate','Dest','ArrTimeh'], right_on=['date','airport','time'])
            # merging destination flight with weather data by common given columns and produce inersection valuea as dest data
            origin_data=origin_data.append(origin_weather, ignore_index=True)
            # append origin weather data to combine 15 airport datas in to single Data frame(origin_data)
            dest_data=dest_data.append(dest_weather, ignore_index=True)
            # append dest​ weather data to combine 15 airport datas in to single Data frame(dest_data)
       

        columns =['FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth', 'DepTime','DepDel15', 'CRSDepTime', 'DepDelayMinutes', 'OriginAirportID','DestAirportID', 'ArrTime', 'CRSArrTime', 'ArrDel15', 'ArrDelayMinutes', 'DepTimeh', 'ArrTimeh']
        flight_weather =pd.merge(origin_data,dest_data,how='inner',on=columns,suffixes=('_Origin','_Dest'))
        #merging origin_data and dest_data with common columns in columns file to produce intersection value as data frame (flight_weather)
        Coln_drop=['date_Origin','time_Origin','Origin','Dest','ArrTimeh','DepTimeh','date_Dest','time_Dest','airport_Origin','airport_Dest']
        flight_weather=flight_weather.drop(Coln_drop,axis=1)
        #drop unwanted columns
        flight_weather=flight_weather.replace('',np.NaN)
        flight_weather=flight_weather.dropna()
        print(year[r]+"_"+month[j])
        #import flight_weather data to .csv for storage with respected name
        #flight_weather.to_csv(mergedata_path+"/flight_weather_data"+year[r]+"_"+month[j]+".csv",index=False)
        final_file=final_file.append(flight_weather, ignore_index=True)
        
final_file.to_csv(mergedata_path+"/final_file_data.csv",index=False)

2016_1
2016_2
2016_3
2016_4
2016_5
2016_6
2016_7
2016_8
2016_9
2016_10
2016_11
2016_12
2017_1
2017_2
2017_3
2017_4
2017_5
2017_6
2017_7
2017_8
2017_9
2017_10
2017_11
2017_12
